In [1]:
import pandas as pd
import pickle
from collections import Counter
import math
import nltk
import requests
import html
import datetime
from datetime import timedelta, date
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import Lasso

%matplotlib inline

In [2]:
data = pd.read_csv("../../dados/resultado.csv", header = None) 

* Data da mensagem
* Usuário que postou a mensagem
* Texto da mensagem
* Pontuação para  classe=paródia
* Pontuação para  classe=opinião
* Pontuação para  classe=campanha
* Pontuação para  classe=experiência pessoal
* Pontuação para  classe=informação
* Resultado (qual a classe foi escolhida pela maior pontuação)
* Informação (JSON) sobre a localização

In [3]:
data.columns = ["date", "user", "message", "0", "1", "2", "3", "4", "class", "location"]

In [4]:
data['date'] = pd.to_datetime(data['date'])

In [5]:
data.dtypes

date        datetime64[ns]
user                object
message             object
0                  float64
1                  float64
2                  float64
3                  float64
4                  float64
class                int64
location            object
dtype: object

In [6]:
data = data[data['date'] < '2011-09-30 23:59:59']

# save pickle of filtered data

# load pickle

In [16]:
with open('D:/Dropbox/A-Marcelo/education-work/2019-ufmg/resultado.pkl', 'rb') as f:
    data = pickle.load(f)

In [17]:
data = data[data['date'] < '2011-09-30 23:59:59']

In [18]:
data['date'] = data['date'].dt.strftime("%Y-%m-%d")

In [19]:
data['date'] = pd.to_datetime(data['date'])

In [20]:
data.tail()

,date,user,message,0,1,2,3,4,class,location
738140,2011-09-12,claricereis91,"@berntfinal Dengue num dá dor na garganta, vc ...",0.147137,0.066065,0.069753,0.544475,0.172570,0,"{u'd': 1460387911967, u'v': 1}"
738141,2011-09-12,guinisinmypants,@DearDany Todos os sintomas de dengue.. Até do...,0.130242,0.059683,0.119528,0.488431,0.202117,0,"{u'd': 1460387911968, u'v': 1}"
738142,2011-09-12,eu_plfl,a @pqjade_ tem problemas e dengue no #cerebro,0.295350,0.144311,0.110989,0.201095,0.248256,1,"{u'city': u'Sao Jose dos Pinhais', u'country':..."
738143,2011-09-12,Pathy_Aleixo,@ronaldoarreios Tô com dengue.... Meu corpo da...,0.084619,0.036423,0.048278,0.739830,0.090849,0,"{u'd': 1460387912416, u'v': 1}"
738144,2011-09-12,_rachelmarques,Minha ta mais parada do que foco de mosquito d...,0.500673,0.088380,0.053413,0.200705,0.156830,1,"{u'city': u'Rio de Janeiro', u'country': u'Bra..."


In [21]:
data.shape

(603588, 10)

### stopwords

In [22]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords = list(set(stopwords))

In [23]:
try: stopwords.remove('não')
except: pass

In [24]:
def remove_stopwords(text):
    tokens = []
    for word in text.lower().split():
        if word not in stopwords:
            tokens.append(word)
    return tokens

new column: tokens without stopwords

In [ ]:
data['message_nostopwords'] = data['message'].apply(remove_stopwords)

### build epidemiologic weeks
* Week one of every year is the first week with 4 days or more.
* Timestamp of the study comprises: 21/11/2010-30/09/2011

In [ ]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2010, 1, 1)
end_dt = date(2011, 10, 30)
dates = []
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))

In [ ]:
dates_2010 = dates[:365]
dates_2011 = dates[365:]

In [ ]:
ep_week_lst = []
count = 0
epidemic_week = 1
ep_week_lst.extend([201000]*2)
for d in dates_2010[2:]:
    if count == 7:
        epidemic_week += 1
        count = 0
    count += 1
    str_week = str(epidemic_week)
    if len(str_week) == 1: str_week = '0'+str_week
    year_week = '2010'+str_week
    ep_week_lst.append(year_week)
    
count = 0
epidemic_week = 1
ep_week_lst.append(201100)
for d in dates_2011[1:]:
    if count == 7:
        epidemic_week += 1
        count = 0
    count += 1
    str_week = str(epidemic_week)
    if len(str_week) == 1: str_week = '0'+str_week
    year_week = '2011'+str_week
    ep_week_lst.append(year_week)

In [ ]:
datetime.datetime.strptime("21/12/2008 20:04:44", "%d/%m/%Y %H:%M:%S").strftime("%Y-%m-%d")

In [ ]:
data.dtypes

In [ ]:
def date_to_epweek(row):
    #print(type(row))
    #print(row)
    row = row.strftime("%Y-%m-%d")
    date_index = dates.index(row)
    year_week = ep_week_lst[date_index]
    return year_week
data['se'] = data['date'].apply(date_to_epweek) #se = epidemiologic week

In [ ]:
data

# Infodengue
The research comprises 12 cities:
* Belém, Belo Horizonte, Brasília,Fortaleza, Manaus, Natal, Rio Branco, Rio de Janeiro, São Paulo, Londrina, Niterói, Ribeirão Preto.
* Source of geocodes: https://ww2.ibge.gov.br/home/geociencias/geografia/redes_fluxos/gestao_do_territorio_2014/base.shtm
* File of the geocodes: Base_de_dados_dos_municipios.xls

### geocodes

In [ ]:
geocodes_BR = pd.read_excel (r'..\..\dados\Base_de_dados_dos_municipios.xls')
geocodes_BR = geocodes_BR[['UF', 'Codmundv', 'NomeMunic']]

### infodengue - Rio de Janeiro

In [ ]:
municipality= 'Rio de Janeiro'.upper()
geocode_row = geocodes_BR.loc[geocodes_BR['NomeMunic'] == municipality]
geocode = list(geocode_row.iloc[:,1])[0]
url = 'https://info.dengue.mat.br/api/alertcity/?geocode=3304557&disease=dengue&format=csv&ew_start=47&ey_start=2010&ew_end=39&ey_end=2011'
r = requests.get(url)
if r:
    print('Success!')
else:
    print('An error has occurred.')

In [ ]:
r.headers

In [ ]:
r = r.text.split('\n')

In [ ]:
count = 0
for row in r:
    split_row = row.split(',')
    r[count] = split_row
    count += 1
headers = r.pop(0)

In [ ]:
df = pd.DataFrame(r, columns=headers)
df = df.iloc[:,2:6]
df.head()

In [ ]:
df.shape

In [ ]:
df['casos'] = df['casos'].fillna(0).astype(int)

In [ ]:
df.groupby(['se']).sum().head(3)

### check epidemiologic week consistency

In [117]:
df.loc[df['data'] == '2011-01-16']

,se,data,tweets,casos
11,201103,2011-01-16,,8
35,201103,2011-01-16,,8
49,201103,2011-01-16,,14
78,201103,2011-01-16,,33
113,201103,2011-01-16,,16
157,201103,2011-01-16,,28
235,201103,2011-01-16,,27
320,201103,2011-01-16,,48
388,201103,2011-01-16,,25
437,201103,2011-01-16,,30


In [118]:
data.loc[data['date'] == '2011-02-16']

,date,user,message,0,1,2,3,4,class,location,message_nostopwords,se
115269,2011-02-16,marconim_,"@_vivianfranca Viroze, muuuuuuuuuuuuuuito fort...",0.268607,0.117382,0.065547,0.399804,0.148660,0,"{u'city': u'Natal', u'country': u'Brasil', u'c...","[@_vivianfranca, viroze,, muuuuuuuuuuuuuuito, ...",201107
115270,2011-02-16,Elyades,Hoje matei um mosquito da dengue com uma palma...,0.365835,0.082711,0.058723,0.300469,0.192262,1,"{u'city': u'Goiania', u'country': u'Brasil', u...","[hoje, matei, mosquito, dengue, palmada, só!, ...",201107
115271,2011-02-16,giuliaportugal_,@EduaardoFranca sera que ta com dengue?,0.187134,0.078271,0.068225,0.414612,0.251759,0,"{u'd': 1460382585083, u'v': 1}","[@eduaardofranca, sera, ta, dengue?]",201107
115272,2011-02-16,May_Angeli,ééé galera.. dengue não é facil não,0.192988,0.051558,0.189787,0.427742,0.137926,0,"{u'city': u'Foz do Iguacu', u'country': u'Bras...","[ééé, galera.., dengue, não, é, facil, não]",201107
115273,2011-02-16,gugugahh,saiindo... muiito sono... amanha aula de music...,0.179459,0.086837,0.162915,0.343563,0.227227,0,"{u'city': u'Jundiai', u'country': u'Brasil', u...","[saiindo..., muiito, sono..., amanha, aula, mu...",201107
115274,2011-02-16,JulianoAPSilva,"RT @pmjponline: Amigos e amigas, vamos denunci...",0.067687,0.086042,0.154552,0.120482,0.571237,3,"{u'city': u'Joao Pessoa', u'country': u'Brasil...","[rt, @pmjponline:, amigos, amigas,, vamos, den...",201107
115275,2011-02-16,baa_marques,RT @gugugahh: saiindo... muiito sono... amanha...,0.179459,0.086837,0.162915,0.343563,0.227227,0,"{u'city': u'Sao Paulo', u'country': u'Brasil',...","[rt, @gugugahh:, saiindo..., muiito, sono..., ...",201107
115276,2011-02-16,baa_marques,mosquito da dengue = eu. Meus amigos sao retar...,0.371380,0.096298,0.091741,0.219992,0.220588,1,"{u'city': u'Sao Paulo', u'country': u'Brasil',...","[mosquito, dengue, =, eu., amigos, sao, retard...",201107
115277,2011-02-16,MaurilioBarbosa,Agora eu enxugo bem a cintura depois do banho....,0.601190,0.108484,0.067611,0.066911,0.155805,1,"{u'country': u'Brasil', u'state': u'MS', u'typ...","[agora, enxugo, bem, cintura, banho., dizem, m...",201107
115278,2011-02-16,raphafinha,Em casa borestando. A unica coisa boa dessa de...,0.129195,0.274254,0.124102,0.321752,0.150697,0,"{u'city': u'Rio de Janeiro', u'country': u'Bra...","[casa, borestando., unica, coisa, boa, dessa, ...",201107


In [62]:
data

,date,user,message,0,1,2,3,4,class,location,message_nostopwords,se
0,2010-11-19 20:04:44,PauloAfBahia,A dengue mata mas com simples gestos de preven...,0.138010,0.101521,0.254248,0.085880,0.420341,3,"{u'd': 1443198862750, u'v': 1}","[dengue, mata, simples, gestos, prevenção, pod...",201046
1,2010-11-19 20:05:05,PortalOuroNegro,Mutirão contra a Dengue começou nesta sexta em...,0.105452,0.056434,0.248333,0.188912,0.400869,3,"{u'city': u'Linhares', u'country': u'Brasil', ...","[mutirão, contra, dengue, começou, nesta, sext...",201046
2,2010-11-19 20:05:30,Kustelaas,@depChicoAlencar amannhã alem do dia da consiê...,0.054194,0.084122,0.213530,0.146927,0.501226,3,"{u'country': u'Brasil', u'state': u'MT', u'typ...","[@depchicoalencar, amannhã, alem, dia, consiên...",201046
3,2010-11-19 20:05:58,Raul_Jungmann,Pernambuco tem risco de sofrer epidemia de den...,0.080289,0.106928,0.054929,0.141276,0.616577,3,"{u'city': u'Recife', u'country': u'Brasil', u'...","[pernambuco, risco, sofrer, epidemia, dengue, ...",201046
4,2010-11-19 20:06:18,laana_mello,Rpz...acho q a Caueira tah com surto de dengue...,0.112013,0.143425,0.094947,0.429626,0.219990,0,"{u'country': u'Brasil', u'state': u'MT', u'typ...","[rpz...acho, q, caueira, tah, surto, dengue,, ...",201046
5,2010-11-19 20:06:30,okcaiolohan,"Dor de Cabeça ha semanas, e meus olhos doem pr...",0.127295,0.045798,0.153509,0.528900,0.144498,0,"{u'd': 1443198862917, u'v': 1}","[dor, cabeça, ha, semanas,, olhos, doem, pra, ...",201046
6,2010-11-19 20:07:05,andreraja,RT @JornaldoBrasil: Novo aliado no combate à d...,0.193595,0.109164,0.183374,0.078817,0.435050,3,"{u'city': u'Rio de Janeiro', u'country': u'Bra...","[rt, @jornaldobrasil:, novo, aliado, combate, ...",201046
7,2010-11-19 20:07:18,PauloAfBahia,LUTA CONTRA A DENGUE!Neste Sábado acontecerá e...,0.098662,0.064887,0.240880,0.082752,0.512820,3,"{u'd': 1443198862855, u'v': 1}","[luta, contra, dengue!neste, sábado, acontecer...",201046
8,2010-11-19 20:07:38,CatchMeJBiebs,"@PoderBelieber há nossa, dengue é horrível.......",0.169313,0.186423,0.081953,0.437851,0.124461,0,"{u'd': 1443198862304, u'v': 1}","[@poderbelieber, nossa,, dengue, é, horrível.....",201046
9,2010-11-19 20:08:51,PoderBelieber,"RT @CatchMeJBiebs: @PoderBelieber há nossa, de...",0.169313,0.186423,0.081953,0.437851,0.124461,0,"{u'd': 1443198862770, u'v': 1}","[rt, @catchmejbiebs:, @poderbelieber, nossa,, ...",201046


### group tweet data by week

In [ ]:
data.groupby(['se']).sum()

### TF-IDF

https://www.kdnuggets.com/2018/08/wtf-tf-idf.html

In [ ]:
idf_counter = Counter()
for text in data['message_nostopwords']:
    #words = text.lower().split()
    #idf_counter.update(words)
    idf_counter.update(text)

In [ ]:
idf = {
  word: math.log(len(data)/df, 2)
  for word, df in idf_counter.items()
}

In [ ]:
def analyze_row(row):
    target_words = row.lower().split()
    tfidf =  {
        word: (1 + math.log(_tf, 2)) * idf[word]
        for word, _tf in Counter(target_words).items()
    }
    num_words = 20
    most_frequent = [
        w  for (w, _) in Counter(target_words).most_common(num_words)
    ]
    sorted_tfidf = [
        w  for (w, _) in sorted(tfidf.items(), key=lambda kv: kv[1], reverse=True)
    ]
    print(row)
    print("Most frequent: {}".format(most_frequent))
    print("Higher TF-IDF: {}".format(sorted_tfidf[:num_words]))

In [ ]:
idf_top = sorted(idf, key=idf.get)[:30]

In [ ]:
for term in idf_top:
    data[term] = 0

In [ ]:
def tfidf(text, term):
    tfidf = 0
    if term in text:
        tf = idf_counter[term]
        tfidf = (1 + math.log(tf, 2)) * idf[term]
        return tfidf
    return tfidf

In [ ]:
for term in idf_top:
    data[term] = data['message'].apply(lambda text: tfidf(text, term))

In [ ]:
sorted(idf, key=idf.get)[-4:]

# To do

### Lasso regression
Using Lasso to choose the most relevant terms

In [86]:
data.head()

,date,user,message,0,1,2,3,4,class,location,...,q,saúde,to,água,pra,dengue!,parada,combater,-,hoje
0,19/11/2010 20:04:44,PauloAfBahia,A dengue mata mas com simples gestos de preven...,0.138010,0.101521,0.254248,0.085880,0.420341,3,"{u'd': 1443198862750, u'v': 1}",...,0.000000,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19/11/2010 20:05:05,PortalOuroNegro,Mutirão contra a Dengue começou nesta sexta em...,0.105452,0.056434,0.248333,0.188912,0.400869,3,"{u'city': u'Linhares', u'country': u'Brasil', ...",...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19/11/2010 20:05:30,Kustelaas,@depChicoAlencar amannhã alem do dia da consiê...,0.054194,0.084122,0.213530,0.146927,0.501226,3,"{u'country': u'Brasil', u'state': u'MT', u'typ...",...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19/11/2010 20:05:58,Raul_Jungmann,Pernambuco tem risco de sofrer epidemia de den...,0.080289,0.106928,0.054929,0.141276,0.616577,3,"{u'city': u'Recife', u'country': u'Brasil', u'...",...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19/11/2010 20:06:18,laana_mello,Rpz...acho q a Caueira tah com surto de dengue...,0.112013,0.143425,0.094947,0.429626,0.219990,0,"{u'country': u'Brasil', u'state': u'MT', u'typ...",...,28.714269,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
data.columns[11:]

Index(['dengue', 'rt', 'é', 'combate', 'contra', 'dengue.', 'não', 'dia',
       'mosquito', 'casos', 'brasil', 'quase', 'ano', 'registra', 'relação',
       'dengue,', 'dobro', 'número', 'ta', 'd', 'q', 'saúde', 'to', 'água',
       'pra', 'dengue!', 'parada', 'combater', '-', 'hoje'],
      dtype='object')

In [91]:
X = data.iloc[:,11:]

In [93]:
X.head()

,dengue,rt,é,combate,contra,dengue.,não,dia,mosquito,casos,...,q,saúde,to,água,pra,dengue!,parada,combater,-,hoje
0,7.906893,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.00000,0.000000,0.0,23.840136,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.00000,21.798437,0.0,0.000000,0.0,0.0,25.643406,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.906893,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7.906893,21.34936,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,28.714269,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X = data

#y = dados da saúde

In [ ]:
lasso = Lasso(alpha=0.4, normalize=True)

# Fit the regressor to the data
lasso.fit(X, y)

# Compute and print the coefficients
lasso_coef = lasso.coef_ 
print(lasso_coef)

# Plot the coefficients
plt.plot(range(len(X)), lasso_coef)
plt.xticks(range(len(X)), ['var 0','var 1','var 2'], rotation=60)
plt.margins(0.02)
plt.show()


# draft

In [ ]:
text_test = [['this', 'is', 'a', 'sample'], ['this', 'is', 'another', 'example']]